In [11]:
from dotenv import load_dotenv
import os

load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")

### 필요한 모듈 로딩

In [12]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM을 통한 요리 정보 해석
- 이미지 -> 맛과 풍미 (image to text)
- 입력 : 요리 이미지(url)
- 출력: 요리명, 요리에 대한 풍미 설명
- 함수로 정의한 다음, RunnableLambda 객체

In [13]:
# 함수 정의 : 이미지 -> 요리명, 풍미 설명 출력
def describe_dish_flavor(input_data):

    prompt = ChatPromptTemplate([
        ("system", """
        You are a culinary expert.
        Based on the input image of a dish,

        you identify the most likely name of the dish by analyzing visual cues such as ingredients, cooking methods, and plating,

        and infer the ingredients and preparation techniques to describe the flavor profile objectively.

        If the dish name is an estimate, this should be stated clearly.
        The flavor description should focus concisely on key taste components (sweetness, saltiness, umami, richness, spiciness, acidity),
        aroma, and textural contrast.
        Subjective judgments and emotional expressions must be excluded, and the explanation should be grounded in observable facts.
        """),
        HumanMessagePromptTemplate.from_template([
            {"text": """아래 이미지의 요리에 대한 요리명과 풍미를 설명해주세요.
             출력형태 :
             요리명 :
             요리의 풍미:
            """},
            {"image_url": "{image_url}"} # image_url는 정해줘 있음.
        ])
    ])

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.1,
        api_key=GOOGLE_API_KEY
    )

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser

    return chain

In [14]:
from langchain_core.runnables import RunnableLambda

# 함수를 전달인자로 넣기
r1 = RunnableLambda(describe_dish_flavor)

# RunnableLambda를 통한 함수 실행
input_data = {
    "image_url": "https://static.wtable.co.kr/image/production/service/recipe/2068/8e90b171-4b7c-44da-affc-bb9bc8297084.jpg?size=800x800"
    }
res = r1.invoke(input_data)

In [15]:
print(res)

요리명 : 궁중떡볶이 (Gungjung Tteokbokki)

요리의 풍미:
이 요리는 간장 기반의 양념으로 볶아낸 떡과 재료들로 구성됩니다. 주된 맛은 간장에서 오는 짭짤함과 은은한 단맛이며, 소고기와 표고버섯에서 깊은 우마미가 느껴집니다. 참기름과 깨가 더해져 고소한 향을 더합니다. 떡은 쫄깃하고 부드러운 식감을 제공하며, 소고기는 부드럽고 촉촉합니다. 당근은 아삭한 식감을, 표고버섯은 쫄깃하면서도 부드러운 식감을 더해 전체적으로 다채로운 질감의 조화를 이룹니다. 매운맛은 거의 없으며, 전반적으로 균형 잡힌 감칠맛과 고소함이 특징입니다.


## 2.  요리에 가장 잘 어울리는 wine top 5 검색
- pinecone 백터 db 저장되어 있음
- index : wine-reviews, namespace: wine-reviews-ns

In [18]:
# 사용자 프롬프트 vector화, 유사도 높은 top 5 찾기
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

# 요리에 대한 풍미 설명) 들어오면,
# 백터 db에 인덱실할 때 사용한 동일 임베딩 모델을 사용해서 임베딩 백터 생성
embedding = OpenAIEmbeddings(
    model = OPENAI_EMBEDDING_MODEL
)
 
# 백터 db에서 유사도 계산, top-3 검색
# 백터 db 객체 생성
vector_db = PineconeVectorStore(
    embedding=embedding,
    index_name=PINECONE_INDEX_NAME,
    namespace=PINECONE_NAMESPACE
)

# 백터 db에서 질문과 가장 유사한, top-5 검색하기
query = res  # 앞서 생성한 요리명, 풍미 설명 결과
print("질문 : ", query)
print("-" *50)
vector_db.similarity_search(query, k=5)

질문 :  요리명 : 궁중떡볶이 (Gungjung Tteokbokki)

요리의 풍미:
이 요리는 간장 기반의 양념으로 볶아낸 떡과 재료들로 구성됩니다. 주된 맛은 간장에서 오는 짭짤함과 은은한 단맛이며, 소고기와 표고버섯에서 깊은 우마미가 느껴집니다. 참기름과 깨가 더해져 고소한 향을 더합니다. 떡은 쫄깃하고 부드러운 식감을 제공하며, 소고기는 부드럽고 촉촉합니다. 당근은 아삭한 식감을, 표고버섯은 쫄깃하면서도 부드러운 식감을 더해 전체적으로 다채로운 질감의 조화를 이룹니다. 매운맛은 거의 없으며, 전반적으로 균형 잡힌 감칠맛과 고소함이 특징입니다.
--------------------------------------------------


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}